### Human in The Loop

In [1]:
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [3]:
from uuid import uuid4

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""
def reduce_messages(left: list[AnyMessage], right: list[AnyMessage]) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged

class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [4]:
tool = TavilySearchResults(max_results=2)

In [5]:
class Agent:
    def __init__(self, model, tools, checkpointer, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges("llm", self.exists_action, {True: "action", False: END})
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer,
            interrupt_before=["action"]
            )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [12]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatOpenAI(model="gpt-3.5-turbo")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [15]:
messages = [HumanMessage(content="Whats the weather in Ankara?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n6OXfxgUZ9o8GkTRdvUmaWzD', 'function': {'arguments': '{"query":"weather in Ankara"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-41eb0bee-54c5-478f-9f41-089a59a27b8e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Ankara'}, 'id': 'call_n6OXfxgUZ9o8GkTRdvUmaWzD', 'type': 'tool_call'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173, 'input_token_details': {'audio': 0, 'cache_read'

In [18]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Ankara?', additional_kwargs={}, response_metadata={}, id='8c8065b7-1a54-4ec4-9850-b1067e2553e6'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n6OXfxgUZ9o8GkTRdvUmaWzD', 'function': {'arguments': '{"query":"weather in Ankara"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-41eb0bee-54c5-478f-9f41-089a59a27b8e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Ankara'}, 'id': 'call_n6OXfxgUZ9o8GkT

In [19]:
abot.graph.get_state(thread).next

('action',)

### continue after interrupt

In [20]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Ankara'}, 'id': 'call_n6OXfxgUZ9o8GkTRdvUmaWzD', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Ankara\', \'region\': \'Ankara\', \'country\': \'Turkey\', \'lat\': 39.9272, \'lon\': 32.8644, \'tz_id\': \'Europe/Istanbul\', \'localtime_epoch\': 1736972982, \'localtime\': \'2025-01-15 23:29\'}, \'current\': {\'last_updated_epoch\': 1736972100, \'last_updated\': \'2025-01-15 23:15\', \'temp_c\': 4.2, \'temp_f\': 39.6, \'is_day\': 0, \'condition\': {\'text\': \'Clear\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/113.png\', \'code\': 1000}, \'wind_mph\': 4.0, \'wind_kph\': 6.5, \'wind_degree\': 196, \'wind_dir\': \'SSW\', \'pressure_mb\': 1020.0, \'pressure_in\': 30.12, \'precip_mm\': 0.01, \'precip_in\': 0.0, \'humidity\': 100, \'cloud\': 0, \'feelslike_c\': 2.6, \'feelslike_f\': 36.8, \'win

In [21]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Ankara?', additional_kwargs={}, response_metadata={}, id='8c8065b7-1a54-4ec4-9850-b1067e2553e6'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n6OXfxgUZ9o8GkTRdvUmaWzD', 'function': {'arguments': '{"query":"weather in Ankara"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-41eb0bee-54c5-478f-9f41-089a59a27b8e-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Ankara'}, 'id': 'call_n6OXfxgUZ9o8GkT

In [22]:
abot.graph.get_state(thread).next

()

In [23]:
messages = [HumanMessage("Who won the 2024 US presidential election?")]
thread = {"configurable": {"thread_id": "44"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TjT29uwWAg3yuGYOi9uHEtKh', 'function': {'arguments': '{"query":"2024 US presidential election winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 156, 'total_tokens': 180, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-074effd7-6dc5-404b-a5b2-722f9b0d169f-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 US presidential election winner'}, 'id': 'call_TjT29uwWAg3yuGYOi9uHEtKh', 'type': 'tool_call'}], usage_metadata={'input_tokens': 156, 'output_tokens': 24, 'total_tokens': 180, 'input_tok

In [31]:
last = abot.graph.get_state(thread)
print(last.values["messages"][-1].content)

Donald Trump won the 2024 US presidential election, defeating Harris to retake the White House. He was elected as the 47th president of the United States in a historic comeback for a second term.


### Modify State

In [32]:
messages = [HumanMessage("Whats the weather in Ankara?")]
thread = {"configurable": {"thread_id": "33"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9w9l47t5k1h1ZSM5M8Sz3LQ5', 'function': {'arguments': '{"query":"weather in Ankara"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-87b995d9-94de-4cfa-a54c-72b68a7801a0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Ankara'}, 'id': 'call_9w9l47t5k1h1ZSM5M8Sz3LQ5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173, 'input_token_details': {'audio': 0, 'cache_read'

In [33]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Ankara?', additional_kwargs={}, response_metadata={}, id='ff0dc778-5113-43ea-ba65-4ecd1c3c6876'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9w9l47t5k1h1ZSM5M8Sz3LQ5', 'function': {'arguments': '{"query":"weather in Ankara"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-87b995d9-94de-4cfa-a54c-72b68a7801a0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Ankara'}, 'id': 'call_9w9l47t5k1h1ZSM

In [34]:
abot.graph.get_state(thread).next

('action',)

In [35]:
current_values = abot.graph.get_state(thread)

In [36]:
current_values.values["messages"][-1]

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9w9l47t5k1h1ZSM5M8Sz3LQ5', 'function': {'arguments': '{"query":"weather in Ankara"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-87b995d9-94de-4cfa-a54c-72b68a7801a0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Ankara'}, 'id': 'call_9w9l47t5k1h1ZSM5M8Sz3LQ5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 152, 'output_tokens': 21, 'total_tokens': 173, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_

In [37]:
current_values.values["messages"][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'weather in Ankara'},
  'id': 'call_9w9l47t5k1h1ZSM5M8Sz3LQ5',
  'type': 'tool_call'}]

In [38]:
_id = current_values.values["messages"][-1].tool_calls[0]["id"]
current_values.values["messages"][-1].tool_calls = [
    {
        'name': 'tavily_search_results_json',
        'args': {'query': 'weather in Kastamonu'},
        'id': _id,
        'type': 'tool_call'
    }
]

In [39]:
abot.graph.update_state(thread, current_values.values)

{'configurable': {'thread_id': '33',
  'checkpoint_ns': '',
  'checkpoint_id': '1efd3f31-5aeb-6da5-8002-7712153df1a4'}}

In [40]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Ankara?', additional_kwargs={}, response_metadata={}, id='ff0dc778-5113-43ea-ba65-4ecd1c3c6876'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9w9l47t5k1h1ZSM5M8Sz3LQ5', 'function': {'arguments': '{"query":"weather in Ankara"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-87b995d9-94de-4cfa-a54c-72b68a7801a0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Kastamonu'}, 'id': 'call_9w9l47t5k1h1

In [41]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Kastamonu'}, 'id': 'call_9w9l47t5k1h1ZSM5M8Sz3LQ5', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Kastamonu\', \'region\': \'Kastamonu\', \'country\': \'Turkey\', \'lat\': 41.3722, \'lon\': 33.7711, \'tz_id\': \'Europe/Istanbul\', \'localtime_epoch\': 1737022704, \'localtime\': \'2025-01-16 13:18\'}, \'current\': {\'last_updated_epoch\': 1737022500, \'last_updated\': \'2025-01-16 13:15\', \'temp_c\': -0.6, \'temp_f\': 30.9, \'is_day\': 1, \'condition\': {\'text\': \'Light sleet\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/317.png\', \'code\': 1204}, \'wind_mph\': 3.8, \'wind_kph\': 6.1, \'wind_degree\': 312, \'wind_dir\': \'NW\', \'pressure_mb\': 1030.0, \'pressure_in\': 30.41, \'precip_mm\': 0.53, \'precip_in\': 0.02, \'humidity\': 97, \'cloud\': 100, \'feelslike_c\': -2.8, \'feelslike_

In [43]:
abot.graph.get_state(thread).values["messages"][-1].content

'I have found the current weather information for Kastamonu, Turkey. The temperature is -0.6°C with light sleet conditions. The wind speed is 6.1 km/h from the northwest. The humidity is 97%.'

In [44]:
messages = [HumanMessage("What should I wear today in Kastamonu")]
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content="Based on the current weather in Kastamonu with a temperature of -0.6°C and light sleet conditions, it is recommended to wear warm and waterproof clothing. Here are some suggestions:\n1. Dress in layers to stay warm.\n2. Wear a waterproof jacket or coat to protect against the sleet.\n3. Wear insulated and waterproof boots to keep your feet dry and warm.\n4. Don't forget a hat, gloves, and a scarf to protect your head, hands, and neck from the cold.\n5. Consider wearing thermal underwear for extra warmth.\n\nIt's important to dress warmly and protect yourself from the cold and wet conditions. Stay safe and comfortable!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 138, 'prompt_tokens': 797, 'total_tokens': 935, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens'

In [45]:
abot.graph.get_state(thread).values["messages"][-1].content

"Based on the current weather in Kastamonu with a temperature of -0.6°C and light sleet conditions, it is recommended to wear warm and waterproof clothing. Here are some suggestions:\n1. Dress in layers to stay warm.\n2. Wear a waterproof jacket or coat to protect against the sleet.\n3. Wear insulated and waterproof boots to keep your feet dry and warm.\n4. Don't forget a hat, gloves, and a scarf to protect your head, hands, and neck from the cold.\n5. Consider wearing thermal underwear for extra warmth.\n\nIt's important to dress warmly and protect yourself from the cold and wet conditions. Stay safe and comfortable!"

### Time Travel

In [46]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print('--')
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Ankara?', additional_kwargs={}, response_metadata={}, id='ff0dc778-5113-43ea-ba65-4ecd1c3c6876'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9w9l47t5k1h1ZSM5M8Sz3LQ5', 'function': {'arguments': '{"query":"weather in Ankara"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-87b995d9-94de-4cfa-a54c-72b68a7801a0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Kastamonu'}, 'id': 'call_9w9l47t5k1h1

In [79]:
last_state = states[-4]

In [80]:
last_state

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Ankara?', additional_kwargs={}, response_metadata={}, id='ff0dc778-5113-43ea-ba65-4ecd1c3c6876'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9w9l47t5k1h1ZSM5M8Sz3LQ5', 'function': {'arguments': '{"query":"weather in Ankara"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-87b995d9-94de-4cfa-a54c-72b68a7801a0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Kastamonu'}, 'id': 'call_9w9l47t5k1h1

In [81]:
for event in abot.graph.stream(None, last_state.config):
    for k, v in event.items():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Kastamonu'}, 'id': 'call_9w9l47t5k1h1ZSM5M8Sz3LQ5', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Kastamonu\', \'region\': \'Kastamonu\', \'country\': \'Turkey\', \'lat\': 41.3722, \'lon\': 33.7711, \'tz_id\': \'Europe/Istanbul\', \'localtime_epoch\': 1737027775, \'localtime\': \'2025-01-16 14:42\'}, \'current\': {\'last_updated_epoch\': 1737027000, \'last_updated\': \'2025-01-16 14:30\', \'temp_c\': -0.9, \'temp_f\': 30.4, \'is_day\': 1, \'condition\': {\'text\': \'Light sleet\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/317.png\', \'code\': 1204}, \'wind_mph\': 4.0, \'wind_kph\': 6.5, \'wind_degree\': 313, \'wind_dir\': \'NW\', \'pressure_mb\': 1030.0, \'pressure_in\': 30.43, \'precip_mm\': 0.56, \'precip_in\': 0.02, \'humidity\': 97, \'cloud\': 100, \'feelslike_c\': -3.3, \'feelslike_

### Go back in time and edit

In [82]:
last_state

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Ankara?', additional_kwargs={}, response_metadata={}, id='ff0dc778-5113-43ea-ba65-4ecd1c3c6876'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9w9l47t5k1h1ZSM5M8Sz3LQ5', 'function': {'arguments': '{"query":"weather in Ankara"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-87b995d9-94de-4cfa-a54c-72b68a7801a0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in Kastamonu'}, 'id': 'call_9w9l47t5k1h1

In [83]:
_id = last_state.values['messages'][-1].tool_calls[0]['id']
last_state.values['messages'][-1].tool_calls = [{'name': 'tavily_search_results_json',
    'args': {'query': 'current weather in Ankara, accuweather'},
    'id': _id}]

In [84]:
branch_state = abot.graph.update_state(last_state.config, last_state.values)

In [85]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Ankara, accuweather'}, 'id': 'call_9w9l47t5k1h1ZSM5M8Sz3LQ5', 'type': 'tool_call'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Ankara\', \'region\': \'Ankara\', \'country\': \'Turkey\', \'lat\': 39.9272, \'lon\': 32.8644, \'tz_id\': \'Europe/Istanbul\', \'localtime_epoch\': 1737025623, \'localtime\': \'2025-01-16 14:07\'}, \'current\': {\'last_updated_epoch\': 1737025200, \'last_updated\': \'2025-01-16 14:00\', \'temp_c\': 8.3, \'temp_f\': 46.9, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 8.9, \'wind_kph\': 14.4, \'wind_degree\': 286, \'wind_dir\': \'WNW\', \'pressure_mb\': 1027.0, \'pressure_in\': 30.33, \'precip_mm\': 0.01, \'precip_in\': 0.0, \'humidity\': 76, \'cloud\': 75, \'feelslike_c\': 5.9, 

## Add message to a state at a given time

In [86]:
last_state

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in Ankara?', additional_kwargs={}, response_metadata={}, id='ff0dc778-5113-43ea-ba65-4ecd1c3c6876'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9w9l47t5k1h1ZSM5M8Sz3LQ5', 'function': {'arguments': '{"query":"weather in Ankara"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 152, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-87b995d9-94de-4cfa-a54c-72b68a7801a0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Ankara, accuweather'}, 'id': 

In [87]:
_id = last_state.values['messages'][-1].tool_calls[0]['id']

In [88]:
state_update = {"messages": [ToolMessage(
    tool_call_id=_id,
    name="tavily_search_results_json",
    content="54 degree celcius",
)]}

In [89]:
branch_and_add = abot.graph.update_state(
    last_state.config, 
    state_update, 
    as_node="action")

In [91]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

{'messages': [AIMessage(content='The current weather in Ankara is 54 degrees Celsius.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 192, 'total_tokens': 205, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3fcea1bd-5160-4456-8a83-bd4cb04823a8-0', usage_metadata={'input_tokens': 192, 'output_tokens': 13, 'total_tokens': 205, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}
